Pipeline:
  1.  Load & Explore Datasets
  2.  Harmonize & Merge
  3.  Preprocessing (Imputation + Encoding)
  4.  Stratified Train/Test Split (80/20)
  5.  SMOTE Oversampling (Training only)
  6.  Feature Scaling
  7.  Ensemble Feature Selection (Chi2 + MI + RFE)  [train-only]
  8.  10-Fold Stratified Cross-Validation
  9.  Test Set Evaluation (7 Classifiers)
  10. Soft Voting Ensemble (RF + HistGB + GB)
  11. Confusion Matrices
  12. Model Comparison Plot
  13. SHAP Explainability
  14. Save all results
"""

In [7]:
# ─────────────────────────────────────────────
# 1. IMPORTS
# ─────────────────────────────────────────────
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (train_test_split, StratifiedKFold,
                                     cross_validate)
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import (chi2, mutual_info_classif,
                                       SelectKBest, RFE)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier,
                               GradientBoostingClassifier,
                               HistGradientBoostingClassifier,
                               VotingClassifier)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score,
                              classification_report, confusion_matrix,
                              ConfusionMatrixDisplay)
from statsmodels.stats.proportion import proportion_confint
import shap

print("=" * 65)
print("  CHRONIC KIDNEY DISEASE — ML PIPELINE")
print("=" * 65)
print("✅ All libraries imported\n")


  CHRONIC KIDNEY DISEASE — ML PIPELINE
✅ All libraries imported



In [8]:
# ─────────────────────────────────────────────
# 2. LOAD DATASETS
# ─────────────────────────────────────────────
print("STEP 1: LOADING DATASETS")
print("-" * 45)

df1 = pd.read_csv("chronic_kidney_disease.csv")
df1.columns = df1.columns.str.strip()
df1.replace('?', np.nan, inplace=True)
print(f"Dataset 1 (UCI CKD)      shape: {df1.shape}")

df2 = pd.read_csv("Chronic_Kidney_Dsease_data.csv")
print(f"Dataset 2 (Extended CKD) shape: {df2.shape}\n")

STEP 1: LOADING DATASETS
---------------------------------------------
Dataset 1 (UCI CKD)      shape: (400, 25)
Dataset 2 (Extended CKD) shape: (1659, 54)



In [9]:
# ─────────────────────────────────────────────
# 3. HARMONIZE & MERGE
# ─────────────────────────────────────────────
print("STEP 2: HARMONIZE & MERGE")
print("-" * 45)

# ── UCI Dataset ──
d1 = df1.copy()
d1['class'] = d1['class'].str.strip().str.replace('\t', '')
d1['target'] = d1['class'].map({'ckd': 1, 'notckd': 0})
d1.drop(columns=['class'], inplace=True)

uci_num = ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc',
           'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc']
uci_cat = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

le = LabelEncoder()
for col in uci_cat:
    if col in d1.columns:
        mask = d1[col].notna() & (d1[col].astype(str) != 'nan')
        vals = d1.loc[mask, col].astype(str)
        if len(vals) > 0:
            le.fit(vals)
            d1.loc[mask, col] = le.transform(vals)
            d1.loc[~mask, col] = np.nan

for col in uci_num:
    if col in d1.columns:
        d1[col] = pd.to_numeric(d1[col], errors='coerce')

d1_fin = d1[uci_num + uci_cat + ['target']].copy()
d1_fin['_src'] = 'uci'

# ── Extended Dataset ──
d2 = df2.copy()
d2_sel = pd.DataFrame()
d2_sel['age']    = d2['Age']
d2_sel['bp']     = d2['SystolicBP']
d2_sel['sc']     = d2['SerumCreatinine']
d2_sel['bu']     = d2['BUNLevels']
d2_sel['hemo']   = d2['HemoglobinLevels']
d2_sel['sod']    = d2['SerumElectrolytesSodium']
d2_sel['pot']    = d2['SerumElectrolytesPotassium']
d2_sel['bgr']    = d2['FastingBloodSugar']
d2_sel['htn']    = d2['FamilyHistoryHypertension'].astype(float)
d2_sel['dm']     = d2['FamilyHistoryDiabetes'].astype(float)
d2_sel['target'] = d2['Diagnosis'].astype(int)
d2_sel['_src']   = 'ext'

for col in d1_fin.columns:
    if col not in d2_sel.columns:
        d2_sel[col] = np.nan

# ── Merge ──
merged = pd.concat([d1_fin, d2_sel], ignore_index=True, sort=False)
merged.drop(columns=['_src'], inplace=True)
merged.to_csv("merged_ckd_dataset.csv", index=False)

print(f"✅ Merged shape : {merged.shape}")
print(f"   CKD     (1) : {(merged['target']==1).sum()}")
print(f"   Not CKD (0) : {(merged['target']==0).sum()}")
print("💾 Saved : merged_ckd_dataset.csv\n")


STEP 2: HARMONIZE & MERGE
---------------------------------------------
✅ Merged shape : (2059, 25)
   CKD     (1) : 1774
   Not CKD (0) : 284
💾 Saved : merged_ckd_dataset.csv



In [11]:
# ─────────────────────────────────────────────
# 4. PREPROCESSING
# ─────────────────────────────────────────────
print("STEP 3: PREPROCESSING")
print("-" * 45)

X = merged.drop(columns=['target'])

# target-এ NaN থাকলে সেই rows drop করো
merged_clean = merged.dropna(subset=['target'])
X = merged_clean.drop(columns=['target'])
y = merged_clean['target'].astype(int)
print(f"Rows dropped (NaN target): {len(merged) - len(merged_clean)}")

imp = SimpleImputer(strategy='median')
X_imp = pd.DataFrame(imp.fit_transform(X), columns=X.columns)
print(f"Features : {X_imp.shape[1]}  |  Records : {X_imp.shape[0]}")
print(f"Missing after imputation : {X_imp.isnull().sum().sum()}\n")

STEP 3: PREPROCESSING
---------------------------------------------
Rows dropped (NaN target): 1
Features : 24  |  Records : 2058
Missing after imputation : 0



In [12]:
# ─────────────────────────────────────────────
# 5. STRATIFIED TRAIN/TEST SPLIT 80/20
# ─────────────────────────────────────────────
print("STEP 4: STRATIFIED TRAIN/TEST SPLIT (80/20)")
print("-" * 45)

X_tr, X_te, y_tr, y_te = train_test_split(
    X_imp, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train : {X_tr.shape[0]}  |  Test : {X_te.shape[0]}")
print(f"Train class dist : {dict(y_tr.value_counts().sort_index())}")
print(f"Test  class dist : {dict(y_te.value_counts().sort_index())}\n")

STEP 4: STRATIFIED TRAIN/TEST SPLIT (80/20)
---------------------------------------------
Train : 1646  |  Test : 412
Train class dist : {0: np.int64(227), 1: np.int64(1419)}
Test  class dist : {0: np.int64(57), 1: np.int64(355)}



In [13]:
# ─────────────────────────────────────────────
# 6. SMOTE (Training only)
# ─────────────────────────────────────────────
print("STEP 5: SMOTE OVERSAMPLING")
print("-" * 45)

X_tr_res, y_tr_res = SMOTE(random_state=42).fit_resample(X_tr, y_tr)
print(f"After SMOTE → {X_tr_res.shape[0]} samples")
print(f"Class dist : {dict(pd.Series(y_tr_res).value_counts().sort_index())}\n")

STEP 5: SMOTE OVERSAMPLING
---------------------------------------------
After SMOTE → 2838 samples
Class dist : {0: np.int64(1419), 1: np.int64(1419)}



In [14]:
# ─────────────────────────────────────────────
# 7. FEATURE SCALING
# ─────────────────────────────────────────────
sc = StandardScaler()
X_tr_sc = pd.DataFrame(sc.fit_transform(X_tr_res), columns=X_tr_res.columns)
X_te_sc  = pd.DataFrame(sc.transform(X_te),        columns=X_te.columns)


In [15]:
# ─────────────────────────────────────────────
# 8. ENSEMBLE FEATURE SELECTION
# ─────────────────────────────────────────────
print("STEP 6: ENSEMBLE FEATURE SELECTION (Chi2 + MI + RFE)")
print("-" * 45)

feats = X_tr_sc.columns.tolist()
k = min(11, len(feats))

# Chi-Square
chi2_sel = SelectKBest(chi2, k=k)
chi2_sel.fit(X_tr_sc - X_tr_sc.min(), y_tr_res)
chi2_f = set(np.array(feats)[chi2_sel.get_support()])

# Mutual Information
mi_sc = mutual_info_classif(X_tr_sc, y_tr_res, random_state=42)
mi_f  = set(np.array(feats)[np.argsort(mi_sc)[::-1][:k]])

# RFE
rfe_sel = RFE(LogisticRegression(max_iter=1000, random_state=42),
              n_features_to_select=k)
rfe_sel.fit(X_tr_sc, y_tr_res)
rfe_f = set(np.array(feats)[rfe_sel.support_])

# Consensus ≥2
votes   = {f: sum([f in chi2_f, f in mi_f, f in rfe_f]) for f in feats}
consens = [f for f, v in votes.items() if v >= 2]

print(f"Chi-Square  ({len(chi2_f)}) : {sorted(chi2_f)}")
print(f"Mutual Info ({len(mi_f)})  : {sorted(mi_f)}")
print(f"RFE         ({len(rfe_f)}) : {sorted(rfe_f)}")
print(f"\n✅ Consensus ({len(consens)}) : {sorted(consens)}\n")

X_tr_sel = X_tr_sc[consens]
X_te_sel  = X_te_sc[consens]

# Plot
plt.figure(figsize=(12, 5))
vs = pd.Series(votes).sort_values(ascending=False)
plt.bar(vs.index, vs.values,
        color=['#2ecc71' if v >= 2 else '#e74c3c' for v in vs.values])
plt.axhline(2, color='navy', ls='--', lw=1.5, label='Threshold ≥ 2')
plt.title('Ensemble Feature Selection – Votes per Feature', fontsize=14)
plt.xlabel('Feature')
plt.ylabel('Votes (out of 3)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('feature_selection_votes.png', dpi=150, bbox_inches='tight')
plt.close()
print("📊 Saved: feature_selection_votes.png")

STEP 6: ENSEMBLE FEATURE SELECTION (Chi2 + MI + RFE)
---------------------------------------------
Chi-Square  (11) : [np.str_('al'), np.str_('ane'), np.str_('appet'), np.str_('bp'), np.str_('cad'), np.str_('dm'), np.str_('htn'), np.str_('pcc'), np.str_('pe'), np.str_('sc'), np.str_('su')]
Mutual Info (11)  : [np.str_('age'), np.str_('bgr'), np.str_('bp'), np.str_('dm'), np.str_('htn'), np.str_('pcv'), np.str_('rbcc'), np.str_('sc'), np.str_('sg'), np.str_('sod'), np.str_('wbcc')]
RFE         (11) : [np.str_('al'), np.str_('appet'), np.str_('bgr'), np.str_('bp'), np.str_('bu'), np.str_('pcv'), np.str_('pe'), np.str_('rbcc'), np.str_('sc'), np.str_('sg'), np.str_('su')]

✅ Consensus (12) : ['al', 'appet', 'bgr', 'bp', 'dm', 'htn', 'pcv', 'pe', 'rbcc', 'sc', 'sg', 'su']

📊 Saved: feature_selection_votes.png


In [16]:
# ─────────────────────────────────────────────
# 9. 10-FOLD STRATIFIED CROSS-VALIDATION
# ─────────────────────────────────────────────
print("\nSTEP 7: 10-FOLD STRATIFIED CROSS-VALIDATION")
print("-" * 45)

models = {
    'Decision Tree':          DecisionTreeClassifier(random_state=42),
    'Random Forest':          RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting':      GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Hist Gradient Boosting': HistGradientBoostingClassifier(random_state=42),
    'SVM':                    SVC(probability=True, random_state=42),
    'KNN':                    KNeighborsClassifier(n_neighbors=5),
    'Logistic Regression':    LogisticRegression(max_iter=1000, random_state=42),
}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_res = {}

for nm, clf in models.items():
    cv = cross_validate(clf, X_tr_sel, y_tr_res, cv=skf,
                        scoring=['accuracy', 'f1', 'roc_auc'])
    cv_res[nm] = {
        'CV Accuracy': round(cv['test_accuracy'].mean(), 4),
        'CV F1':       round(cv['test_f1'].mean(), 4),
        'CV AUC-ROC':  round(cv['test_roc_auc'].mean(), 4),
    }
    print(f"  {nm:28s}| Acc:{cv_res[nm]['CV Accuracy']:.4f} "
          f"| F1:{cv_res[nm]['CV F1']:.4f} "
          f"| AUC:{cv_res[nm]['CV AUC-ROC']:.4f}")

cv_df = pd.DataFrame(cv_res).T




STEP 7: 10-FOLD STRATIFIED CROSS-VALIDATION
---------------------------------------------
  Decision Tree               | Acc:0.8939 | F1:0.8930 | AUC:0.8939
  Random Forest               | Acc:0.9263 | F1:0.9272 | AUC:0.9781
  Gradient Boosting           | Acc:0.8989 | F1:0.9030 | AUC:0.9646
  Hist Gradient Boosting      | Acc:0.9179 | F1:0.9185 | AUC:0.9742
  SVM                         | Acc:0.8735 | F1:0.8823 | AUC:0.9450
  KNN                         | Acc:0.8862 | F1:0.8852 | AUC:0.9591
  Logistic Regression         | Acc:0.8111 | F1:0.8195 | AUC:0.9002


In [17]:
# ─────────────────────────────────────────────
# 10. TEST SET EVALUATION
# ─────────────────────────────────────────────
print("\nSTEP 8: TEST SET EVALUATION")
print("-" * 45)

te_res  = {}
trained = {}

for nm, clf in models.items():
    clf.fit(X_tr_sel, y_tr_res)
    trained[nm] = clf
    yp   = clf.predict(X_te_sel)
    yprb = clf.predict_proba(X_te_sel)[:, 1]

    acc = accuracy_score(y_te, yp)
    f1  = f1_score(y_te, yp)
    auc = roc_auc_score(y_te, yprb)
    te_res[nm] = {'Test Accuracy': acc, 'Test F1': f1, 'Test AUC-ROC': auc}
    print(f"  {nm:28s}| Acc:{acc:.4f} | F1:{f1:.4f} | AUC:{auc:.4f}")

te_df = pd.DataFrame(te_res).T

best_nm = te_df['Test Accuracy'].idxmax()
best_ac = te_df.loc[best_nm, 'Test Accuracy']
nc      = int(round(best_ac * len(y_te)))
ci_l, ci_h = proportion_confint(nc, len(y_te), method='wilson')
print(f"\n✅ Best: {best_nm} | Acc:{best_ac:.4f} | 95% CI:[{ci_l:.3f},{ci_h:.3f}]\n")


STEP 8: TEST SET EVALUATION
---------------------------------------------
  Decision Tree               | Acc:0.8762 | F1:0.9268 | AUC:0.7883
  Random Forest               | Acc:0.8981 | F1:0.9412 | AUC:0.8708
  Gradient Boosting           | Acc:0.9078 | F1:0.9472 | AUC:0.8759
  Hist Gradient Boosting      | Acc:0.8859 | F1:0.9339 | AUC:0.8585
  SVM                         | Acc:0.9126 | F1:0.9497 | AUC:0.8619
  KNN                         | Acc:0.8519 | F1:0.9112 | AUC:0.8079
  Logistic Regression         | Acc:0.8374 | F1:0.9007 | AUC:0.8782

✅ Best: SVM | Acc:0.9126 | 95% CI:[0.881,0.936]



In [36]:
# ─────────────────────────────────────────────
# 11. SOFT VOTING ENSEMBLE
# ─────────────────────────────────────────────
print("STEP 9: SOFT VOTING ENSEMBLE (SVM + HistGB + GB)")
print("-" * 45)

# SVM probability=True দিতে হবে soft voting-এর জন্য
ens = VotingClassifier(estimators=[
    ('SVM', SVC(probability=True, random_state=42)),
    ('HGB', HistGradientBoostingClassifier(random_state=42)),
    ('GB',  GradientBoostingClassifier(n_estimators=100, random_state=42)),
], voting='soft')

ens.fit(X_tr_sel, y_tr_res)
yp_e  = ens.predict(X_te_sel)
ypr_e = ens.predict_proba(X_te_sel)[:, 1]

ea = accuracy_score(y_te, yp_e)
ef = f1_score(y_te, yp_e)
eu = roc_auc_score(y_te, ypr_e)

print(f"  Ensemble | Acc:{ea:.4f} | F1:{ef:.4f} | AUC:{eu:.4f}")
print("\nClassification Report:")
print(classification_report(y_te, yp_e, target_names=['Not CKD', 'CKD']))


STEP 9: SOFT VOTING ENSEMBLE (SVM + HistGB + GB)
---------------------------------------------
  Ensemble | Acc:0.9078 | F1:0.9471 | AUC:0.8705

Classification Report:
              precision    recall  f1-score   support

     Not CKD       0.69      0.60      0.64        57
         CKD       0.94      0.96      0.95       355

    accuracy                           0.91       412
   macro avg       0.82      0.78      0.79       412
weighted avg       0.90      0.91      0.90       412



In [37]:
# ─────────────────────────────────────────────
# 12. CONFUSION MATRICES
# ─────────────────────────────────────────────
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
ax = axes.flatten()

for i, (nm, clf) in enumerate(trained.items()):
    yp = clf.predict(X_te_sel)
    ConfusionMatrixDisplay(confusion_matrix(y_te, yp),
                           display_labels=['Not CKD', 'CKD']).plot(
        ax=ax[i], colorbar=False, cmap='Blues')
    ax[i].set_title(f"{nm}\nAcc:{accuracy_score(y_te,yp):.4f}", fontsize=9)

ConfusionMatrixDisplay(confusion_matrix(y_te, yp_e),
                       display_labels=['Not CKD', 'CKD']).plot(
    ax=ax[7], colorbar=False, cmap='Greens')
ax[7].set_title(f"Soft Voting Ensemble\nAcc:{ea:.4f}", fontsize=9)

plt.suptitle('Confusion Matrices — All Models', fontsize=15)
plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.close()
print("📊 Saved: confusion_matrices.png")

📊 Saved: confusion_matrices.png


In [26]:
# ─────────────────────────────────────────────
# 13. MODEL COMPARISON PLOT
# ─────────────────────────────────────────────
all_r = te_df.copy()
all_r.loc['Soft Voting Ensemble'] = [ea, ef, eu]

fig, ax = plt.subplots(figsize=(13, 6))
x = np.arange(len(all_r))
w = 0.25
ax.bar(x - w, all_r['Test Accuracy'], w, label='Accuracy',  color='#3498db')
ax.bar(x,     all_r['Test F1'],       w, label='F1 Score',  color='#e67e22')
ax.bar(x + w, all_r['Test AUC-ROC'],  w, label='AUC-ROC',   color='#27ae60')
ax.set_xticks(x)
ax.set_xticklabels(all_r.index, rotation=30, ha='right', fontsize=9)
ax.set_ylim(0.8, 1.02)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance — Test Set', fontsize=14)
ax.legend(); ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print("📊 Saved: model_comparison.png")

📊 Saved: model_comparison.png


In [33]:
# ─────────────────────────────────────────────
# 14. SHAP EXPLAINABILITY
# ─────────────────────────────────────────────
print("\nSTEP 10: SHAP EXPLAINABILITY")
print("-" * 45)

explainer = shap.TreeExplainer(trained['Hist Gradient Boosting'])
sv = explainer.shap_values(X_te_sel)

if isinstance(sv, list):
    sv = sv[1]
elif hasattr(sv, 'ndim') and sv.ndim == 3:
    sv = sv[:, :, 1]

fl = list(consens)

# Beeswarm
plt.figure(figsize=(10, 6))
shap.summary_plot(sv, X_te_sel, feature_names=fl, show=False, plot_type='dot')
plt.title('SHAP Summary Plot — Hist Gradient Boosting (CKD)', fontsize=13)
plt.tight_layout()
plt.savefig('shap_summary.png', dpi=150, bbox_inches='tight')
plt.close()
print("📊 Saved: shap_summary.png")

# Bar
plt.figure(figsize=(9, 5))
shap.summary_plot(sv, X_te_sel, feature_names=fl, show=False, plot_type='bar')
plt.title('SHAP Feature Importance', fontsize=13)
plt.tight_layout()
plt.savefig('shap_bar.png', dpi=150, bbox_inches='tight')
plt.close()
print("📊 Saved: shap_bar.png")

shap_imp = pd.DataFrame({
    'Feature':     fl,
    'Mean |SHAP|': np.abs(sv).mean(axis=0)
}).sort_values('Mean |SHAP|', ascending=False)

print("\nTop Features by SHAP:")
print(shap_imp.to_string(index=False))


STEP 10: SHAP EXPLAINABILITY
---------------------------------------------
📊 Saved: shap_summary.png
📊 Saved: shap_bar.png

Top Features by SHAP:
Feature  Mean |SHAP|
     sc     1.206513
    pcv     1.179048
     sg     0.837398
    bgr     0.825889
     bp     0.722170
   rbcc     0.507404
     dm     0.494335
     al     0.411935
    htn     0.399728
  appet     0.119590
     pe     0.029165
     su     0.000000


In [38]:
# ─────────────────────────────────────────────
# FINAL SUMMARY
# ─────────────────────────────────────────────
print("\n" + "=" * 65)
print("  FINAL SUMMARY")
print("=" * 65)
print("\n📋 CV Results:")
print(cv_df.to_string())
print("\n📋 Test Results:")
print(all_r.round(4).to_string())
print(f"\n🏆 Best Model    : {best_nm}")
print(f"   Accuracy      : {best_ac:.4f} | 95% CI:[{ci_l:.3f},{ci_h:.3f}]")

print(f"\n🏆 Ensemble (SVM + HistGB + GB)")

print(f"   Acc:{ea:.4f} | F1:{ef:.4f} | AUC:{eu:.4f}")
print(f"\n🔑 Top-5 SHAP Features:")
for _, r in shap_imp.head(5).iterrows():
    print(f"   {r['Feature']:15s} → {r['Mean |SHAP|']:.4f}")

print("""
✅ Pipeline complete! Output files:
   📄 merged_ckd_dataset.csv
   📊 feature_selection_votes.png
   📊 confusion_matrices.png
   📊 model_comparison.png
   📊 shap_summary.png
   📊 shap_bar.png
   📄 cv_results.csv
   📄 test_results.csv
   📄 shap_importance.csv
""")


  FINAL SUMMARY

📋 CV Results:
                        CV Accuracy   CV F1  CV AUC-ROC
Decision Tree                0.8939  0.8930      0.8939
Random Forest                0.9263  0.9272      0.9781
Gradient Boosting            0.8989  0.9030      0.9646
Hist Gradient Boosting       0.9179  0.9185      0.9742
SVM                          0.8735  0.8823      0.9450
KNN                          0.8862  0.8852      0.9591
Logistic Regression          0.8111  0.8195      0.9002

📋 Test Results:
                        Test Accuracy  Test F1  Test AUC-ROC
Decision Tree                  0.8762   0.9268        0.7883
Random Forest                  0.8981   0.9412        0.8708
Gradient Boosting              0.9078   0.9472        0.8759
Hist Gradient Boosting         0.8859   0.9339        0.8585
SVM                            0.9126   0.9497        0.8619
KNN                            0.8519   0.9112        0.8079
Logistic Regression            0.8374   0.9007        0.8782
Soft Voting Ens